In [1]:
#TODO: add shell commands

In [2]:
#imports
import requests
import re
from bs4 import BeautifulSoup
from kafka import KafkaProducer
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
import json

In [3]:
#set parameters
source_url='https://öffi.at/?archive=1&text=&types=2%2C3&page='
kafka_server = 'localhost:9092'

In [4]:
#get number of total pages
response = requests.get(source_url)
soup = BeautifulSoup(response.content, 'html.parser')
pages=int(soup.find(string=re.compile(r'Aktuelle Seite: \d+/\d+')).split('/')[-1].strip()[:-1])
print(f'{pages} pages found in total')

3118 pages found in total


In [5]:
#init producer
producer = KafkaProducer(bootstrap_servers=kafka_server,value_serializer=lambda v: json.dumps(v).encode('utf-8'))

In [6]:
#get data

#WARN: öffi.at limits us to a small number of results per request
#currently we just send a single request for testing purposes
#TODO: implement multithreading before actually running it on the full dataset

#for n in range(1,pages+1):
response = requests.get(source_url)
soup = BeautifulSoup(response.content, 'html.parser')
#find the list of all delays on this page, then iterate through it's elements
toplist=soup.find('ul',{'class':'category-filter'})
for delay in toplist.findChildren('li',attrs={'class': 'disruption uk-padding-small'},recursive=False):
    lines=[]
    stations=[]
    result={}
    #assign variables
    id=delay.attrs['id']
    title=delay.find('h2',{'class':'disruption-title'}).text.split(':')[-1].strip()
    content=delay.find('div',{'class':'uk-accordion-content'})
    for line in content.find_all('ul')[0].find_all('li'):
        lines.append(line.text)
    for station in content.find_all('ul')[1].find_all('li'):
        stations.append(station.text)
    start=content.find_all(string=re.compile(r': \d{2}\.\d{2}\.\d{4} \d{2}:\d{2}'))[0].split(': ')[1]
    end=content.find_all(string=re.compile(r': \d{2}\.\d{2}\.\d{4} \d{2}:\d{2}'))[1].split(': ')[1]
    #combine into a single dict and send to kafka
    result={'id':id,'title':title,'lines':lines,'stations':stations,'start':start,'end':end}
    producer.send('delays', result)
    producer.flush()

In [7]:
producer.close()